In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
#import cPickle as pkl
import _pickle as cPickle
import time
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import glob
%matplotlib inline  
print ("Packages loaded")

# Load dataset

In [ ]:
dirpath = "/home/saurabh/Documents/data/iccv09Data/images/"
#logs_path = "/home/saurabh/Documents/data/iccv09Data/images/"
height = 240
width  = 320
resize_ratio = 4
nr_img = 0
fileList = glob.glob(dirpath + '*.jpg')
for i, file in enumerate(fileList):
    img = Image.open(file)
    array = np.array(img) 
    if array.shape[0] == height and array.shape[1] == width:
        nr_img = nr_img + 1
        rgb = array.reshape(1, height, width, 3)
        imglow = img.resize((int(width/resize_ratio)
                ,int(height/resize_ratio)), Image.BICUBIC)
        imglow = imglow.resize((width, height), Image.BICUBIC)
        rgblow = np.array(np.float32(imglow)/255.)
        rgblow = rgblow.reshape(1, height, width, 3)
        rgb = np.reshape(rgb, [1, -1])
        rgblow = np.reshape(rgblow, [1, -1])
        if nr_img == 1:
            data = rgb
            datalow = rgblow
        else:
            data = np.concatenate((data, rgb), axis=0)
            datalow = np.concatenate((datalow, rgblow), axis=0)
        
print ("nr_img is %d" % (nr_img))
print ("Shape of 'data' is %s" % (data.shape,))
print ("Shape of 'datalow' is %s" % (datalow.shape,))

# Divide into two sets
## (xtrain, ytrain) and (xtest, ytest)

In [ ]:
randidx = np.random.permutation(nr_img)
nrtrain = int(nr_img*0.7)
nrtest  = nr_img - nrtrain
xtrain  = datalow[randidx[0:nrtrain], :]
ytrain  = data[randidx[0:nrtrain], :]
xtest   = datalow[randidx[nrtrain:nr_img], :]
ytest   = data[randidx[nrtrain:nr_img], :]
print ("Shape of 'xtrain' is %s" % (xtrain.shape,))
print ("Shape of 'ytrain' is %s" % (ytrain.shape,))
print ("Shape of 'xtest' is %s" % (xtest.shape,))
print ("Shape of 'ytest' is %s" % (ytest.shape,))

# Plot some images

In [ ]:
# Train
randidx = np.random.randint(nrtrain)
currx = xtrain[randidx, :]
currx = np.reshape(currx, [height, width, 3])
plt.imshow(currx)
plt.title("Train input image")
plt.show()
curry = ytrain[randidx, :]
curry = np.reshape(curry, [height, width, 3])
plt.imshow(curry)
plt.title("Train output image")
plt.show() 
# Test
randidx = np.random.randint(nrtest)
currx = xtest[randidx, :]
currx = np.reshape(currx, [height, width, 3])
plt.imshow(currx)
plt.title("Test input image")
plt.show()
curry = ytest[randidx, :]
curry = np.reshape(curry, [height, width, 3])
plt.imshow(curry)
plt.title("Test output image")
plt.show()

# Define network

In [ ]:
n1 = 32
n2 = 64
n3 = 64
n4 = 64
n5 = 64
n6 = 3
ksize = 3
weights = {
    'ce1': tf.Variable(tf.random_normal([ksize, ksize, 3, n1],  stddev=0.01)),
    'ce2': tf.Variable(tf.random_normal([ksize, ksize, n1, n2], stddev=0.01)),
    'ce3': tf.Variable(tf.random_normal([ksize, ksize, n2, n3], stddev=0.01)),
    'ce4': tf.Variable(tf.random_normal([ksize, ksize, n3, n4], stddev=0.01)),
    'ce5': tf.Variable(tf.random_normal([ksize, ksize, n4, n5], stddev=0.01)),
    'ce6': tf.Variable(tf.random_normal([ksize, ksize, n5, n6],  stddev=0.01))
}
biases = {
    'be1': tf.Variable(tf.random_normal([n1], stddev=0.01)),
    'be2': tf.Variable(tf.random_normal([n2], stddev=0.01)),
    'be3': tf.Variable(tf.random_normal([n3], stddev=0.01)),
    'be4': tf.Variable(tf.random_normal([n4], stddev=0.01)),
    'be5': tf.Variable(tf.random_normal([n5], stddev=0.01)),
    'be6': tf.Variable(tf.random_normal([n6], stddev=0.01))
}
def srn(_X, _W, _b, _keepprob):
    _input_r = tf.reshape(_X, shape=[-1, height, width, 3])
    # Encoder
    _ce1 = tf.nn.relu(tf.add(tf.nn.conv2d(_input_r, _W['ce1']
        , strides=[1, 1, 1, 1], padding='SAME'), _b['be1']))
    _ce1 = tf.nn.dropout(_ce1, _keepprob)
    _ce2 = tf.nn.relu(tf.add(tf.nn.conv2d(_ce1, _W['ce2']
        , strides=[1, 1, 1, 1], padding='SAME'), _b['be2'])) 
    _ce2 = tf.nn.dropout(_ce2, _keepprob)
    _ce3 = tf.nn.relu(tf.add(tf.nn.conv2d(_ce2, _W['ce3']
        , strides=[1, 1, 1, 1], padding='SAME'), _b['be3'])) 
    _ce3 = tf.nn.dropout(_ce3, _keepprob)
    
    _ce4 = tf.nn.relu(tf.add(tf.nn.conv2d(_ce3, _W['ce4']
        , strides=[1, 1, 1, 1], padding='SAME'), _b['be4']))
    _ce4 = tf.nn.dropout(_ce4, _keepprob)
    _ce5 = tf.nn.relu(tf.add(tf.nn.conv2d(_ce4, _W['ce5']
        , strides=[1, 1, 1, 1], padding='SAME'), _b['be5'])) 
    _ce5 = tf.nn.dropout(_ce5, _keepprob)
    _ce6 = tf.nn.relu(tf.add(tf.nn.conv2d(_ce5, _W['ce6']
        , strides=[1, 1, 1, 1], padding='SAME'), _b['be6'])) 
    _out = _ce6 + _input_r
    return {'input_r': _input_r, 'ce1': _ce1, 'ce2': _ce2, 'ce3': _ce3
        , 'ce4': _ce4, 'ce5': _ce5, 'ce6': _ce6
        , 'layers': (_input_r, _ce1, _ce2, _ce3, _ce4, _ce5, _ce6)
        , 'out': _out}
print ("Network ready")

# Define functions

In [ ]:
dim = height*width*3
x = tf.placeholder(tf.float32, [None, dim])
y = tf.placeholder(tf.float32, [None, dim])
keepprob = tf.placeholder(tf.float32)
pred = srn(x, weights, biases, keepprob)['out']
cost = tf.reduce_mean(tf.square(srn(x, weights, biases, keepprob)['out'] 
            - tf.reshape(y, shape=[-1, height, width, 3])))

merged_summary_op = tf.summary.merge_all()


learning_rate = 0.001
optm = tf.train.AdamOptimizer(learning_rate, 0.9).minimize(cost)
init = tf.initialize_all_variables()
print ("Functions ready")
tf.summary.scalar("loss", cost)
merged_summary_op = tf.summary.merge_all()
#summary_writer = tf.summary.FileWriter('/tmp/tensorflow_logs', graph=sess.graph)


# Run

In [ ]:
sess = tf.Session()
sess.run(init)
# Fit all training data
batch_size = 1
n_epochs   = 1000000
print("Strart training..")
randidx = np.random.randint(nrtrain, size=batch_size)
batch_xs = xtrain[randidx, :]
batch_ys = ytrain[randidx, :]
train_error = []
test_error = []

summary_writer = tf.summary.FileWriter('/tmp/tensorflow_logs',sess.graph)
#tf.global_variables_initializer

#summary_writer = tf.summary.FileWriter('/tmp/tensorflow_logs', graph=tf.get_default_graph())


for epoch_i in range(n_epochs):
    total_batch = int(nrtrain // batch_size)
    for batch_i in range(nrtrain // batch_size):
        randidx = np.random.randint(nrtrain, size=batch_size)
        batch_xs = xtrain[randidx, :]
        batch_ys = ytrain[randidx, :]
        sess.run(optm, feed_dict={x: batch_xs
            , y: batch_ys, keepprob: 0.7})
        
        #summary = sess.run([merged_summary_op], feed_dict={x: train_xs, y: train_ys, keepprob: 1.})
        #summary_writer.add_summary(summary, epoch_i)
        
        
        #summary = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
        
        #summary_writer.add_summary(summary, epoch_i * total_batch + batch_i)
    
    if (epoch_i % 100) == 0:
        n_examples = 2
        print ("Training dataset")
        randidx = np.random.randint(nrtrain, size=n_examples)
        train_xs = xtrain[randidx, :]
        train_ys = ytrain[randidx, :]
        recon = sess.run(pred, feed_dict={x: train_xs, keepprob: 1.})
        print ("[%02d/%02d] Training cost: %.4f" % (epoch_i, n_epochs
            , sess.run(cost, feed_dict={x: train_xs
            , y: train_ys, keepprob: 1.})))
        
        
        
        c = sess.run(cost, feed_dict={x: train_xs , y: train_ys, keepprob: 1.})
        train_error.append(c)
        
        #c = sess.run([cost],feed_dict={x: train_xs, y: train_ys})
        
        #summary_writer.add_summary(summary, cost, epoch_i, n_epochs)
        
        #fig, axs = plt.subplots(3, n_examples, figsize=(15, 20))
        #for example_i in range(n_examples):
         #   axs[0][example_i].imshow(np.reshape(
              #  train_xs[example_i, :], (height, width, 3)))
            #axs[1][example_i].imshow(np.reshape(
               # recon[example_i, :], (height, width, 3)))
            #axs[2][example_i].imshow(np.reshape(
                #train_ys[example_i, :], (height, width, 3)))
        
        
        #plt.show()
        print ("Test dataset")
        randidx = np.random.randint(nrtest, size=n_examples)
        test_xs = xtest[randidx, :]
        test_ys = ytest[randidx, :]
        recon = sess.run(pred, feed_dict={x: test_xs, keepprob: 1.})
        print ("[%02d/%02d] Test cost: %.4f" % (epoch_i, n_epochs
            , sess.run(cost, feed_dict={x: test_xs
            , y: test_ys, keepprob: 1.})))
        c2 = sess.run(cost, feed_dict={x: test_xs , y: test_ys, keepprob: 1.})
        test_error.append(c2)

        #fig, axs = plt.subplots(3, n_examples, figsize=(15, 20))
        #for example_i in range(n_examples):
            #axs[0][example_i].imshow(np.reshape(
                #test_xs[example_i, :], (height, width, 3)))
            #axs[1][example_i].imshow(np.reshape(
                #recon[example_i, :], (height, width, 3)))
            #axs[2][example_i].imshow(np.reshape(
                #test_ys[example_i, :], (height, width, 3)))
        plt.show()
print("Training done. ")
print(train_error)
print(test_error)
print ("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Strart training..
Training dataset
[00/1000000] Training cost: 13819.9375
Test dataset
[00/1000000] Test cost: 22676.9805
Training dataset
[100/1000000] Training cost: 995.1548
Test dataset
[100/1000000] Test cost: 1338.0475
